In [2]:
# ============================================================
# Notebook setup
# ============================================================

%load_ext autoreload
%autoreload 2

figsize=(14, 4)

from utils import DataAggregator
import pandas as pd
# TODO: Consider creating just one column TeamForm instead of home and away team form.

# Feature engineering

In order to more accurately predict the games, we now want to perform some feature engineering in order to create new variables that can be used for the games. But first of all, lets again take a look at the dataset we have available.

In [3]:
data_aggregator = DataAggregator()
df = data_aggregator.get_data(["E0"])

In [4]:
df.columns

Index(['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG',
       'HTAG', 'HTR', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC',
       'AC', 'HY', 'AY', 'HR', 'AR', 'B365H', 'B365D', 'B365A'],
      dtype='object')

In [5]:
df.head()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A
0,E0,13/08/05,Aston Villa,Bolton,2,2,D,2,2,D,...,16,7,8,0,2,0,0,2.30,3.25,3.00
1,E0,13/08/05,Everton,Man United,0,2,A,0,1,A,...,14,8,6,3,1,0,0,5.00,3.40,1.72
2,E0,13/08/05,Fulham,Birmingham,0,0,D,0,0,D,...,13,6,6,1,2,0,0,2.37,3.25,2.87
3,E0,13/08/05,Man City,West Brom,0,0,D,0,0,D,...,11,3,6,2,3,0,0,1.72,3.40,5.00
4,E0,13/08/05,Middlesbrough,Liverpool,0,0,D,0,0,D,...,11,5,0,2,3,1,0,2.87,3.20,2.40


We know that if we want to use this data to predict using classifiers and so on, we need to make the data numerical. Lets start by making the date numerical.

In [6]:
df = data_aggregator.format_date(df, "Date")

The format date function extracts the year, month, day and day of week from the original dataframe and ensures it is sorted by the oldest dates first.

In [7]:
df.head()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,AY,HR,AR,B365H,B365D,B365A,Year,Month,Day,DayOfWeek
0,E0,2002-08-17,Blackburn,Sunderland,0,0,D,0,0,D,...,2,0,0,1.727,3.25,4.333,2002,8,17,5
1,E0,2002-08-17,Southampton,Middlesbrough,0,0,D,0,0,D,...,0,0,0,2.250,3.25,2.750,2002,8,17,5
2,E0,2002-08-17,Man United,West Brom,1,0,H,0,0,D,...,1,0,1,1.200,5.00,12.000,2002,8,17,5
3,E0,2002-08-17,Charlton,Chelsea,2,3,A,2,1,H,...,3,1,0,2.800,3.25,2.200,2002,8,17,5
4,E0,2002-08-17,Fulham,Bolton,4,1,H,3,1,H,...,2,0,0,1.727,3.25,4.333,2002,8,17,5


Furthermore, we want encode the result in a numerical format. We will use the following mapping:
- ``H``-> 1
- ``D``-> 0
- ``A``> -1

In [8]:
df = data_aggregator.encode_result(df, 
                                   mapping={"H": 1, "D": 0, "A": -1}, 
                                   result_column="FTR")

In [9]:
df.head()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,AY,HR,AR,B365H,B365D,B365A,Year,Month,Day,DayOfWeek
0,E0,2002-08-17,Blackburn,Sunderland,0,0,0,0,0,D,...,2,0,0,1.727,3.25,4.333,2002,8,17,5
1,E0,2002-08-17,Southampton,Middlesbrough,0,0,0,0,0,D,...,0,0,0,2.250,3.25,2.750,2002,8,17,5
2,E0,2002-08-17,Man United,West Brom,1,0,1,0,0,D,...,1,0,1,1.200,5.00,12.000,2002,8,17,5
3,E0,2002-08-17,Charlton,Chelsea,2,3,-1,2,1,H,...,3,1,0,2.800,3.25,2.200,2002,8,17,5
4,E0,2002-08-17,Fulham,Bolton,4,1,1,3,1,H,...,2,0,0,1.727,3.25,4.333,2002,8,17,5


Next up, we also want to encode some form data for the teams as well. We want to encode the form each team has for the last 5 games. This can be done using the mean outcome of the previous 5 games.

In [11]:
form_window = 5

df['ShiftedFTR_Home'] = df.groupby('HomeTeam')['FTR'].shift(1)
df['ShiftedFTR_Away'] = df.groupby('AwayTeam')['FTR'].shift(1)

home_team_win_from = df.groupby('HomeTeam')['ShiftedFTR_Home'].rolling(window=form_window).apply(lambda x: (x == 1).sum() / form_window).reset_index(0, drop=True)
away_team_win_from = df.groupby('AwayTeam')['ShiftedFTR_Home'].rolling(window=form_window).apply(lambda x: (x == -1).sum() / form_window).reset_index(0, drop=True)

df = pd.concat([df, home_team_win_from.rename('HomeTeamWinForm'), away_team_win_from.rename('AwayTeamWinForm')], axis=1)

df = df.drop(columns=['ShiftedFTR_Home', 'ShiftedFTR_Away'])

In [12]:
df[["HomeTeam", "AwayTeam", "FTR", "HomeTeamWinForm", "AwayTeamWinForm"]]

,HomeTeam,AwayTeam,FTR,HomeTeamWinForm,AwayTeamWinForm
0,Blackburn,Sunderland,0,NaN,NaN
1,Southampton,Middlesbrough,0,NaN,NaN
2,Man United,West Brom,1,NaN,NaN
3,Charlton,Chelsea,-1,NaN,NaN
4,Fulham,Bolton,1,NaN,NaN
...,...,...,...,...,...
8435,Bournemouth,Arsenal,1,0.4,0.0
8436,Man United,Brentford,1,0.4,0.4
8437,Wolves,Man City,-1,0.2,0.4
8438,Liverpool,Chelsea,1,0.8,0.4


We can see from the DataFrame above that we now have some missing data. We need to consider how to fill them. Should we just remove them, or should we instead fill them with a selected value.

We have decided to subsitute the ``NaN``values with the mean of each teams win form.

In [13]:
df["HomeTeamWinForm"] = df["HomeTeamWinForm"].fillna(df.groupby("HomeTeam")["HomeTeamWinForm"].transform("mean"))
df["AwayTeamWinForm"] = df["AwayTeamWinForm"].fillna(df.groupby("AwayTeam")["AwayTeamWinForm"].transform("mean"))
df[["HomeTeam", "AwayTeam", "FTR", "HomeTeamWinForm", "AwayTeamWinForm"]]

,HomeTeam,AwayTeam,FTR,HomeTeamWinForm,AwayTeamWinForm
0,Blackburn,Sunderland,0,0.415135,0.275336
1,Southampton,Middlesbrough,0,0.356981,0.267606
2,Man United,West Brom,1,0.687770,0.331405
3,Charlton,Chelsea,-1,0.415556,0.298544
4,Fulham,Bolton,1,0.426403,0.303333
...,...,...,...,...,...
8435,Bournemouth,Arsenal,1,0.400000,0.000000
8436,Man United,Brentford,1,0.400000,0.400000
8437,Wolves,Man City,-1,0.200000,0.400000
8438,Liverpool,Chelsea,1,0.800000,0.400000


We can also have a look at the average points per game (PPG) a team has.

In [14]:
# Map the 'FTR' values to points for the home and away teams
df['HomePoints'] = df['FTR'].map({1: 3, 0: 1, -1: 0})
df['AwayPoints'] = df['FTR'].map({1: 0, 0: 1, -1: 3})

# Shift the points to calculate rolling averages for past games
df['ShiftedHomePoints'] = df.groupby('HomeTeam')['HomePoints'].shift(1)
df['ShiftedAwayPoints'] = df.groupby('AwayTeam')['AwayPoints'].shift(1)

# Calculate rolling points per game
home_team_ppg = df.groupby('HomeTeam')['ShiftedHomePoints'].rolling(window=form_window).mean().reset_index(0, drop=True)
away_team_ppg = df.groupby('AwayTeam')['ShiftedAwayPoints'].rolling(window=form_window).mean().reset_index(0, drop=True)

# Add the new PPG columns to the DataFrame
df = pd.concat([df, home_team_ppg.rename('HomeTeamPPG'), away_team_ppg.rename('AwayTeamPPG')], axis=1)

# Drop intermediate columns
df = df.drop(columns=['HomePoints', 'AwayPoints', 'ShiftedHomePoints', 'ShiftedAwayPoints'])

Yet again, we decide to fill the null values with the mean for each team.

In [15]:
df["HomeTeamPPG"] = df["HomeTeamPPG"].fillna(df.groupby("HomeTeam")["HomeTeamPPG"].transform("mean"))
df["AwayTeamPPG"] = df["AwayTeamPPG"].fillna(df.groupby("AwayTeam")["AwayTeamPPG"].transform("mean"))

In [16]:
df.head()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,B365D,B365A,Year,Month,Day,DayOfWeek,HomeTeamWinForm,AwayTeamWinForm,HomeTeamPPG,AwayTeamPPG
0,E0,2002-08-17,Blackburn,Sunderland,0,0,0,0,0,D,...,3.25,4.333,2002,8,17,5,0.415135,0.275336,1.524324,0.729148
1,E0,2002-08-17,Southampton,Middlesbrough,0,0,0,0,0,D,...,3.25,2.750,2002,8,17,5,0.356981,0.267606,1.374340,0.805442
2,E0,2002-08-17,Man United,West Brom,1,0,1,0,0,D,...,5.00,12.000,2002,8,17,5,0.687770,0.331405,2.245564,0.800000
3,E0,2002-08-17,Charlton,Chelsea,2,3,-1,2,1,H,...,3.25,2.200,2002,8,17,5,0.415556,0.298544,1.475556,1.785612
4,E0,2002-08-17,Fulham,Bolton,4,1,1,3,1,H,...,3.25,4.333,2002,8,17,5,0.426403,0.303333,1.487129,0.951351


# END TEST

Next up, we also want to consider the goal form of each team. Lets create a column HomeTeamGoalForm and AwayTeamGoalForm.

In [12]:
df['ShiftedFTHG_Home'] = df.groupby('HomeTeam')['FTHG'].shift(1)
df['ShiftedFTAG_Away'] = df.groupby('AwayTeam')['FTAG'].shift(1)

home_team_goal_form = df.groupby('HomeTeam')['ShiftedFTHG_Home'].rolling(window=form_window).mean().reset_index(0, drop=True)
away_team_goal_form = df.groupby('AwayTeam')['ShiftedFTAG_Away'].rolling(window=form_window).mean().reset_index(0, drop=True)

df = pd.concat([df, home_team_goal_form.rename('HomeTeamGoalForm'), away_team_goal_form.rename('AwayTeamGoalForm')], axis=1)

df = df.drop(columns=['ShiftedFTHG_Home', 'ShiftedFTAG_Away'])

In [13]:
df[["HomeTeam", "AwayTeam", "FTR", "HomeTeamGoalForm", "AwayTeamGoalForm"]]

,HomeTeam,AwayTeam,FTR,HomeTeamGoalForm,AwayTeamGoalForm
0,Blackburn,Sunderland,0,NaN,NaN
1,Southampton,Middlesbrough,0,NaN,NaN
2,Man United,West Brom,1,NaN,NaN
3,Charlton,Chelsea,-1,NaN,NaN
4,Fulham,Bolton,1,NaN,NaN
...,...,...,...,...,...
8435,Bournemouth,Arsenal,1,1.6,1.8
8436,Man United,Brentford,1,0.8,0.8
8437,Wolves,Man City,-1,1.4,2.4
8438,Liverpool,Chelsea,1,2.2,3.0


Again we need to fill in the missing values. For this we will again use the mean of each teams average amount of goals scored.

In [14]:
df["HomeTeamGoalForm"] = df["HomeTeamGoalForm"].fillna(df.groupby("HomeTeam")["HomeTeamGoalForm"].transform("mean"))
df["AwayTeamGoalForm"] = df["AwayTeamGoalForm"].fillna(df.groupby("AwayTeam")["AwayTeamGoalForm"].transform("mean"))
df[["HomeTeam", "AwayTeam", "FTR", "HomeTeamGoalForm", "AwayTeamGoalForm"]]

,HomeTeam,AwayTeam,FTR,HomeTeamGoalForm,AwayTeamGoalForm
0,Blackburn,Sunderland,0,1.354595,0.879821
1,Southampton,Middlesbrough,0,1.382642,0.838095
2,Man United,West Brom,1,2.039329,0.865289
3,Charlton,Chelsea,-1,1.351111,1.606715
4,Fulham,Bolton,1,1.379538,1.004324
...,...,...,...,...,...
8435,Bournemouth,Arsenal,1,1.600000,1.800000
8436,Man United,Brentford,1,0.800000,0.800000
8437,Wolves,Man City,-1,1.400000,2.400000
8438,Liverpool,Chelsea,1,2.200000,3.000000


Next up, we would also like to model the goals conceded for each team.

In [15]:
df['ShiftedFTAG_Home'] = df.groupby('HomeTeam')['FTAG'].shift(1)
df['ShiftedFTHG_Away'] = df.groupby('AwayTeam')['FTHG'].shift(1)

home_team_goal_against_form = df.groupby('HomeTeam')['FTAG'].rolling(window=form_window).mean().reset_index(0, drop=True)
away_team_goal_against_form = df.groupby('AwayTeam')['FTHG'].rolling(window=form_window).mean().reset_index(0, drop=True)

df = pd.concat([df, home_team_goal_against_form.rename('HomeTeamGoalAgainstForm'), away_team_goal_against_form.rename('AwayTeamGoalAgainstForm')], axis=1)

df = df.drop(columns=['ShiftedFTAG_Home', 'ShiftedFTHG_Away'])

In [16]:
df[["HomeTeam", "AwayTeam", "FTR", "HomeTeamGoalAgainstForm", "AwayTeamGoalAgainstForm"]]

,HomeTeam,AwayTeam,FTR,HomeTeamGoalAgainstForm,AwayTeamGoalAgainstForm
0,Blackburn,Sunderland,0,NaN,NaN
1,Southampton,Middlesbrough,0,NaN,NaN
2,Man United,West Brom,1,NaN,NaN
3,Charlton,Chelsea,-1,NaN,NaN
4,Fulham,Bolton,1,NaN,NaN
...,...,...,...,...,...
8435,Bournemouth,Arsenal,1,1.0,0.8
8436,Man United,Brentford,1,1.8,2.0
8437,Wolves,Man City,-1,3.0,0.6
8438,Liverpool,Chelsea,1,0.4,1.0


Yet again, we need to fill in the missing values.

In [17]:
df["HomeTeamGoalAgainstForm"] = df["HomeTeamGoalAgainstForm"].fillna(df.groupby("HomeTeam")["HomeTeamGoalAgainstForm"].transform("mean"))
df["AwayTeamGoalAgainstForm"] = df["AwayTeamGoalAgainstForm"].fillna(df.groupby("AwayTeam")["AwayTeamGoalAgainstForm"].transform("mean"))
df[["HomeTeam", "AwayTeam", "FTR", "HomeTeamGoalAgainstForm", "AwayTeamGoalAgainstForm"]]

,HomeTeam,AwayTeam,FTR,HomeTeamGoalAgainstForm,AwayTeamGoalAgainstForm
0,Blackburn,Sunderland,0,1.151613,1.750000
1,Southampton,Middlesbrough,0,1.300752,1.493243
2,Man United,West Brom,1,0.812919,1.651029
3,Charlton,Chelsea,-1,1.340659,1.055024
4,Fulham,Bolton,1,1.317105,1.688172
...,...,...,...,...,...
8435,Bournemouth,Arsenal,1,1.000000,0.800000
8436,Man United,Brentford,1,1.800000,2.000000
8437,Wolves,Man City,-1,3.000000,0.600000
8438,Liverpool,Chelsea,1,0.400000,1.000000


Instead of having to manually do this again, we have created a function in the ``DataAggregator``class that we can run to generate all form data. This is called ``create_form_data``.

Lets now have a look at what we have done so far:

In [18]:
df = data_aggregator.get_data(["E0"])
df = data_aggregator.format_date(df, "Date")
df = data_aggregator.encode_result(df,
                                   mapping={"H": 1, "D": 0, "A": -1}, 
                                   result_column="FTR")
df = data_aggregator.create_form_data(df, form_window=5)

In [19]:
df

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,Year,Month,Day,DayOfWeek,HomeTeamWinForm,AwayTeamWinForm,HomeTeamGoalForm,AwayTeamGoalForm,HomeTeamGoalAgainstForm,AwayTeamGoalAgainstForm
0,E0,2002-08-17,Blackburn,Sunderland,0,0,0,0,0,D,...,2002,8,17,5,0.415135,0.275336,1.354595,0.879821,1.151613,1.750000
1,E0,2002-08-17,Southampton,Middlesbrough,0,0,0,0,0,D,...,2002,8,17,5,0.356981,0.267606,1.382642,0.838095,1.300752,1.493243
2,E0,2002-08-17,Man United,West Brom,1,0,1,0,0,D,...,2002,8,17,5,0.687770,0.331405,2.039329,0.865289,0.812919,1.651029
3,E0,2002-08-17,Charlton,Chelsea,2,3,-1,2,1,H,...,2002,8,17,5,0.415556,0.298544,1.351111,1.606715,1.340659,1.055024
4,E0,2002-08-17,Fulham,Bolton,4,1,1,3,1,H,...,2002,8,17,5,0.426403,0.303333,1.379538,1.004324,1.317105,1.688172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8435,E0,2024-10-19,Bournemouth,Arsenal,2,0,1,0,0,D,...,2024,10,19,5,0.400000,0.000000,1.600000,1.800000,1.000000,0.800000
8436,E0,2024-10-19,Man United,Brentford,2,1,1,0,1,A,...,2024,10,19,5,0.400000,0.400000,0.800000,0.800000,1.800000,2.000000
8437,E0,2024-10-20,Wolves,Man City,1,2,-1,1,1,D,...,2024,10,20,6,0.200000,0.400000,1.400000,2.400000,3.000000,0.600000
8438,E0,2024-10-20,Liverpool,Chelsea,2,1,1,1,0,H,...,2024,10,20,6,0.800000,0.400000,2.200000,3.000000,0.400000,1.000000


Lastly, in order to create models that we easily can use to predict the matches, we need to encode the team names. We can either one hot encode the teams with boolean values [Team Name]Home/AwayTeam or ordinal encode the teams with a index. We will choose the first option.

In [20]:
df = pd.get_dummies(df, columns=["HomeTeam", "AwayTeam"], prefix=["HomeTeam", "AwayTeam"])

In [21]:
df.head()

,Div,Date,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HS,...,AwayTeam_Southampton,AwayTeam_Stoke,AwayTeam_Sunderland,AwayTeam_Swansea,AwayTeam_Tottenham,AwayTeam_Watford,AwayTeam_West Brom,AwayTeam_West Ham,AwayTeam_Wigan,AwayTeam_Wolves
0,E0,2002-08-17,0,0,0,0,0,D,D Elleray,15,...,False,False,True,False,False,False,False,False,False,False
1,E0,2002-08-17,0,0,0,0,0,D,B Knight,12,...,False,False,False,False,False,False,False,False,False,False
2,E0,2002-08-17,1,0,1,0,0,D,S Bennett,20,...,False,False,False,False,False,False,True,False,False,False
3,E0,2002-08-17,2,3,-1,2,1,H,G Barber,5,...,False,False,False,False,False,False,False,False,False,False
4,E0,2002-08-17,4,1,1,3,1,H,A Wiley,13,...,False,False,False,False,False,False,False,False,False,False


Now that we have completed our feature engineering. Lets create a function ``preprocess_data`` in the DataAggregator class that we can use to create all this data. We will also filter out all columns that we wont be using to predict the matches.

In [22]:
df = data_aggregator.preprocess_data(data_aggregator.get_data(["E0"]), 
                                     date_column="Date",
                                     home_team_column="HomeTeam",
                                     away_team_column="AwayTeam",
                                     result_column="FTR",
                                     form_window=5)
df

,Date,FTR,B365H,B365D,B365A,Year,Month,Day,DayOfWeek,HomeTeamWinForm,...,AwayTeam_Southampton,AwayTeam_Stoke,AwayTeam_Sunderland,AwayTeam_Swansea,AwayTeam_Tottenham,AwayTeam_Watford,AwayTeam_West Brom,AwayTeam_West Ham,AwayTeam_Wigan,AwayTeam_Wolves
0,2002-08-17,0,1.727,3.25,4.333,2002,8,17,5,0.415135,...,False,False,True,False,False,False,False,False,False,False
1,2002-08-17,0,2.250,3.25,2.750,2002,8,17,5,0.356981,...,False,False,False,False,False,False,False,False,False,False
2,2002-08-17,1,1.200,5.00,12.000,2002,8,17,5,0.687770,...,False,False,False,False,False,False,True,False,False,False
3,2002-08-17,-1,2.800,3.25,2.200,2002,8,17,5,0.415556,...,False,False,False,False,False,False,False,False,False,False
4,2002-08-17,1,1.727,3.25,4.333,2002,8,17,5,0.426403,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8435,2024-10-19,1,5.000,3.90,1.650,2024,10,19,5,0.400000,...,False,False,False,False,False,False,False,False,False,False
8436,2024-10-19,1,1.700,4.20,4.500,2024,10,19,5,0.400000,...,False,False,False,False,False,False,False,False,False,False
8437,2024-10-20,-1,8.000,6.00,1.330,2024,10,20,6,0.200000,...,False,False,False,False,False,False,False,False,False,False
8438,2024-10-20,1,1.620,4.10,5.000,2024,10,20,6,0.800000,...,False,False,False,False,False,False,False,False,False,False


Lastly, we will also create a new column GD, which will stand for goal difference, and will be the result of FTHG-FTAG.

In [24]:
df = data_aggregator.get_data(["E0"])

df["GD"] = df["FTHG"] - df["FTAG"]
df.head()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,GD
0,E0,13/08/05,Aston Villa,Bolton,2,2,D,2,2,D,...,7,8,0,2,0,0,2.30,3.25,3.00,0
1,E0,13/08/05,Everton,Man United,0,2,A,0,1,A,...,8,6,3,1,0,0,5.00,3.40,1.72,-2
2,E0,13/08/05,Fulham,Birmingham,0,0,D,0,0,D,...,6,6,1,2,0,0,2.37,3.25,2.87,0
3,E0,13/08/05,Man City,West Brom,0,0,D,0,0,D,...,3,6,2,3,0,0,1.72,3.40,5.00,0
4,E0,13/08/05,Middlesbrough,Liverpool,0,0,D,0,0,D,...,5,0,2,3,1,0,2.87,3.20,2.40,0


This we will put into a new function in the ``DataAggregator``class, named ``create_gd_feature``.